In [1]:
activities_list = """attach a camera to a tripod
boil water
chop an onion
clean up broken glass
cleaning bathtub
fill a bucket in a small sink
folding piece of cloth
freeze pies
hanging up bedsheets
make a steak
make a strawberry slushie
melt white chocolate
mixing drinks
mowing the lawn
putting away Halloween decorations
putting away toys
putting up shelves
setting the fire
spraying for bugs
thawing frozen food""".replace(" ", "_").replace("-", "_").replace("'", "_").split("\n")
activities = set(activities_list)

## Load list of available objects

In [6]:
# Are there any categories that are missing from the table?
import pathlib, glob, json, os
from collections import defaultdict
# available_scenes = {"house_single_floor"}
available_scenes = [
    "Beechwood_0_int",
    "Beechwood_1_int",
    "Benevolence_0_int",
    "Benevolence_1_int",
    "Benevolence_2_int",
    "Ihlen_0_int",
    "Ihlen_1_int",
    "Merom_0_int",
    "Merom_1_int",
    "Pomaria_0_int",
    "Pomaria_1_int",
    "Pomaria_2_int",
    "Rs_int",
    "Wainscott_0_int",
    "Wainscott_1_int",
]
providers = {}
objects_by_category = defaultdict(list)
object_lists = glob.glob(r"D:\BEHAVIOR-1K\asset_pipeline\cad\*\*\artifacts\object_list.json")
scene_object_lists = [os.path.join(r"D:\BEHAVIOR-1K\asset_pipeline\cad\scenes", scene, r"artifacts\object_list.json") for scene in available_scenes]
for olf in object_lists + scene_object_lists:
    dirname = pathlib.Path(olf).parts[-4] + "/" + pathlib.Path(olf).parts[-3]
    with open(olf, "r") as f:
        ol = json.load(f)
    # if "scenes" not in olf:
    #     pass
        # Some filters for object files
        # if not ol["success"]:
        #     continue
        # if len(ol["provided_objects"]) > 10:
        #     continue
    for obj in ol["provided_objects"]:
        providers[obj] = dirname
        objects_by_category[obj.split("-")[0]].append(obj)

In [10]:
len(providers)

7293

In [11]:
provided_categories = {x.split("-")[0] for x in providers.keys()}

In [13]:
provided_categories

{'latte',
 'lime',
 'cracker_box',
 'carpet',
 'mousepad',
 'champagne_bottle',
 'dehumidifier',
 'canvas',
 'cilantro',
 'bag_of_popcorn',
 'green_onion',
 'thumbtack',
 'mask',
 'cream',
 'ball',
 'paprika',
 'bowl',
 'bar_cart',
 'mulch',
 'camera',
 'geode',
 'console_table',
 'fragrance_decanter',
 'weed',
 'dental_floss',
 'cactus',
 'balsamic_vinegar',
 'cioppino',
 'paper_liners',
 'egg_carton',
 'tire',
 'twine',
 'smoker',
 'boxing_gloves',
 'citrus',
 'cranberry',
 'bacon',
 'leaf_blower',
 'spray_paint',
 'croissant',
 'can',
 'orange',
 'breakfast_table',
 'funnel',
 'jelly_bean',
 'gin_bottle',
 'blinds',
 'kitchen_analog_scale',
 'duster',
 'rock',
 'turtleneck',
 'scrambled_eggs',
 'bookcase',
 'outlet',
 'blouse',
 'penny',
 'shelf_side',
 'map',
 'quail_breast',
 'socks',
 'cigar',
 'pineapple',
 'pop_case',
 'wine_bottle',
 'sugar',
 'onion',
 'salmon',
 'fruit_punch',
 'holly',
 'adhesive',
 'truck',
 'plug',
 'parsley',
 'rail_fence',
 'coffee_filter',
 'raspberry'

## Load list of task-required synsets

In [14]:
import json, csv
from nltk.corpus import wordnet as wn
import networkx as nx

# Build the legit-synset graph
G = nx.DiGraph()
G.add_nodes_from(x.name() for x in wn.all_synsets())
for parent in wn.all_synsets():
    for child in parent.hyponyms():
        G.add_edge(parent.name(), child.name())
        
# Add the illegit-synset graph
with open(r"D:\BEHAVIOR-1K\asset_pipeline\metadata\custom_synsets.csv") as f:
    reader = csv.DictReader(f)
    for row in reader:
        child = row["custom_synset"].strip()
        parent = wn.synset(row["hypernyms"].strip()).name()
        assert parent in G.nodes, "Could not find " + parent
        G.add_edge(parent, child)
        
legit_synsets = set(G.nodes)

In [17]:
def canonicalize(s):
    try:
        return wn.synset(s).name()
    except:
        return s

In [18]:
with open(r"D:\BEHAVIOR-1K\asset_pipeline\metadata\b200_objects.json", "r") as f:
    obj_mapping = json.load(f)
    task_required_synsets_by_activity = {k: [canonicalize(x) for x in obj_mapping[k] if x != "agent.n.01"] for k in activities}
task_required_synsets = {x for objs in task_required_synsets_by_activity.values() for x in objs}
synset_requiring_tasks = {s: [t for t, ss in task_required_synsets_by_activity.items() if s in ss] for s in task_required_synsets}

In [19]:
task_required_synsets

{'atomizer.n.01',
 'basket.n.01',
 'bathtub.n.01',
 'black_pepper.n.02',
 'blender.n.01',
 'bottle.n.01',
 'bowl.n.01',
 'broken__glass.n.02',
 'broom.n.01',
 'bucket.n.01',
 'cabinet.n.01',
 'caldron.n.01',
 'camera.n.01',
 'candle.n.01',
 'carton.n.02',
 'chopping_board.n.01',
 'clothesline.n.01',
 'clove.n.03',
 'cocktail.n.01',
 'cola.n.02',
 'countertop.n.01',
 'cup.n.01',
 'date.n.08',
 'diced__onion.n.03',
 'dryer.n.01',
 'dustpan.n.02',
 'electric_refrigerator.n.01',
 'fireplace.n.01',
 'firewood.n.01',
 'fish.n.02',
 'floor.n.01',
 'glass.n.02',
 'grass.n.01',
 'ice_cube.n.01',
 'insectifuge.n.01',
 'juice.n.01',
 'kettle.n.01',
 'knife.n.01',
 'lawn.n.01',
 'lawn_mower.n.01',
 'lemon_juice.n.01',
 'lemon_juice__bottle.n.01',
 'lighter.n.02',
 'melted__white_chocolate.n.01',
 'newspaper.n.03',
 'olive.n.04',
 'olive_oil.n.01',
 'onion.n.03',
 'oven.n.01',
 'pan.n.01',
 'pepper_mill.n.01',
 'pie.n.01',
 'plaything.n.01',
 'pumpkin.n.02',
 'salt.n.02',
 'saltshaker.n.01',
 'sauc

In [20]:
illegal_task_required_synsets = {x for x in task_required_synsets - legit_synsets}
print(illegal_task_required_synsets)

{'diced__onion.n.03', 'melted__white_chocolate.n.01', 'broken__glass.n.02'}


In [21]:
legit_task_required_synsets = (task_required_synsets & legit_synsets)

In [22]:
print("\n".join(sorted(legit_task_required_synsets)))

atomizer.n.01
basket.n.01
bathtub.n.01
black_pepper.n.02
blender.n.01
bottle.n.01
bowl.n.01
broom.n.01
bucket.n.01
cabinet.n.01
caldron.n.01
camera.n.01
candle.n.01
carton.n.02
chopping_board.n.01
clothesline.n.01
clove.n.03
cocktail.n.01
cola.n.02
countertop.n.01
cup.n.01
date.n.08
dryer.n.01
dustpan.n.02
electric_refrigerator.n.01
fireplace.n.01
firewood.n.01
fish.n.02
floor.n.01
glass.n.02
grass.n.01
ice_cube.n.01
insectifuge.n.01
juice.n.01
kettle.n.01
knife.n.01
lawn.n.01
lawn_mower.n.01
lemon_juice.n.01
lemon_juice__bottle.n.01
lighter.n.02
newspaper.n.03
olive.n.04
olive_oil.n.01
onion.n.03
oven.n.01
pan.n.01
pepper_mill.n.01
pie.n.01
plaything.n.01
pumpkin.n.02
salt.n.02
saltshaker.n.01
saucepot.n.01
scrub_brush.n.01
sheet.n.03
shelf.n.01
sink.n.01
smoothie.n.02
soap.n.01
sofa.n.01
stain.n.01
steak.n.01
stove.n.01
strawberry.n.01
table.n.02
tree.n.01
tripod.n.01
tupperware.n.01
vodka.n.01
water.n.06
white_chocolate.n.01


## Load list of synsets in our dataset

In [23]:
import csv
# Get the category - synset mapping
pairs = {}
synset_to_cat = defaultdict(list)
with open(r"D:\BEHAVIOR-1K\asset_pipeline\metadata\category_mapping.csv", newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        category = row["category"].strip()
        synset = row["synset"].strip()
        if not synset or not category:
            print(f"Skipping problematic row: {row}")
            continue
        if category not in provided_categories:
            continue
        synset = canonicalize(synset)
        pairs[category] = synset
        synset_to_cat[synset].append(category)
found_synsets = set(pairs.values())
found_categories = set(pairs.keys())

In [24]:
# What are the categories that for some reason are not added to the spreadsheet?
provided_categories - found_categories

{'acetone',
 'acid',
 'adhseive_strip',
 'air_filter',
 'air_freshner',
 'alfredo_sauce',
 'algae',
 'aluminium_foil',
 'ammonia',
 'applesauce',
 'arepa',
 'ash',
 'beans',
 'bedhseet',
 'bin',
 'bird_feeder',
 'bleach',
 'blow_dryer',
 'bowl_of_salad',
 'broccolini',
 'bug_repellent',
 'cactus',
 'calendar',
 'can',
 'caraway_seeds',
 'cash_box',
 'cashew',
 'casserole_dish',
 'catalog',
 'caulk',
 'cd',
 'ceiling',
 'centerpiece',
 'chainsaw',
 'chalkboard',
 'champagne',
 'charger',
 'cheesecake',
 'chicken_bouillon',
 'chicken_tender',
 'chicken_wire',
 'chickpeas',
 'chips',
 'chocolate',
 'chocolate_biscuit',
 'chocolate_spread',
 'chopblock',
 'chopsticks',
 'cilantro',
 'cioppino',
 'clam',
 'cleaner',
 'clothesline_pole',
 'cocktail',
 'copper_wire',
 'cranberry_juice_bottle',
 'dog_food_bowl',
 'doll',
 'duster',
 'dvd',
 'fork',
 'ice_tray',
 'icicle_lights',
 'kabob',
 'key',
 'kitchen_scales',
 'label',
 'lacquer',
 'latte',
 'lavender_oil',
 'leaf',
 'leaf_blower',
 'lea

In [25]:
found_invalid_synsets = {x for x in found_synsets if x not in legit_synsets and canonicalize(x) not in legit_synsets}
print("\n".join(found_invalid_synsets))

olive_oil__bottle.n.01
coffee cup.n.01
sugar__jar.n.01
shelf_top.n.01
pop__case.n.02
liquid_soap__bottle.n.01
cream__box.n.02
shelf_shelf.n.01
shelf_baseboard.n.01
shelf_back.n.01
half_apple.n.01
oatmeal__box.n.01
cracker__box.n.01
chocolate__box.n.02
cranberry__box.n.02
shelf_side.n.01


# STOP!
**Fix everything before this before continuing.**

In [26]:
for act in activities_list:
    required = task_required_synsets_by_activity[act]
    found = set()
    for s in required:
        if s not in legit_synsets:
            continue
        child_synsets = [s] + list(nx.descendants(G, s))
        for cs in child_synsets:
            if cs in found_synsets:
                found.add(s)
                break
                
    missing = set(required) - found
    # print("1" if not missing else 0) 
    print(act, "--", ", ".join(sorted(missing)))
    

attach_a_camera_to_a_tripod -- 
boil_water -- water.n.06
chop_an_onion -- diced__onion.n.03
clean_up_broken_glass -- broken__glass.n.02
cleaning_bathtub -- stain.n.01
fill_a_bucket_in_a_small_sink -- water.n.06
folding_piece_of_cloth -- 
freeze_pies -- water.n.06
hanging_up_bedsheets -- 
make_a_steak -- black_pepper.n.02, olive_oil.n.01
make_a_strawberry_slushie -- lemon_juice.n.01, smoothie.n.02
melt_white_chocolate -- melted__white_chocolate.n.01
mixing_drinks -- cocktail.n.01, cola.n.02, vodka.n.01
mowing_the_lawn -- 
putting_away_Halloween_decorations -- 
putting_away_toys -- 
putting_up_shelves -- 
setting_the_fire -- 
spraying_for_bugs -- insectifuge.n.01
thawing_frozen_food -- 


In [27]:
# How many of the required synsets exist:
from collections import defaultdict
found_task_required_synsets = set()
found_task_required_objs_by_synset = defaultdict(set)
for s in legit_task_required_synsets:
    # Get the tree of the synset
    child_synsets = [s] + list(nx.descendants(G, s))
    for cs in child_synsets:
        if cs in found_synsets:
            found_task_required_synsets.add(s)  # Only add the sought-after synset, not children
            
            s_cats = synset_to_cat[cs]
            s_objs = {obj for cat in s_cats for obj in objects_by_category[cat]}
            found_task_required_objs_by_synset[s].update(s_objs)

found_task_required_objects = {cat for cats in found_task_required_objs_by_synset.values() for cat in cats}
not_found_task_required_synsets = legit_task_required_synsets - found_task_required_synsets
print(len(found_task_required_synsets), "found")
print(len(not_found_task_required_synsets), "not found")
print(not_found_task_required_synsets)

62 found
10 not found
{'smoothie.n.02', 'cocktail.n.01', 'vodka.n.01', 'olive_oil.n.01', 'water.n.06', 'lemon_juice.n.01', 'insectifuge.n.01', 'cola.n.02', 'stain.n.01', 'black_pepper.n.02'}


In [28]:
# What synsets match the most crap?
sorted([(-len(objs), s) for s, objs in found_task_required_objs_by_synset.items()])

[(-202, 'floor.n.01'),
 (-198, 'table.n.02'),
 (-120, 'cabinet.n.01'),
 (-111, 'bottle.n.01'),
 (-107, 'shelf.n.01'),
 (-84, 'plaything.n.01'),
 (-58, 'tree.n.01'),
 (-46, 'glass.n.02'),
 (-41, 'cup.n.01'),
 (-40, 'bowl.n.01'),
 (-39, 'basket.n.01'),
 (-36, 'sofa.n.01'),
 (-31, 'sink.n.01'),
 (-22, 'knife.n.01'),
 (-18, 'chopping_board.n.01'),
 (-18, 'firewood.n.01'),
 (-18, 'pan.n.01'),
 (-17, 'electric_refrigerator.n.01'),
 (-17, 'oven.n.01'),
 (-13, 'bathtub.n.01'),
 (-13, 'candle.n.01'),
 (-13, 'steak.n.01'),
 (-12, 'saucepot.n.01'),
 (-12, 'sheet.n.03'),
 (-12, 'stove.n.01'),
 (-11, 'dryer.n.01'),
 (-10, 'clove.n.03'),
 (-10, 'countertop.n.01'),
 (-9, 'ice_cube.n.01'),
 (-9, 'onion.n.03'),
 (-8, 'juice.n.01'),
 (-8, 'pie.n.01'),
 (-6, 'carton.n.02'),
 (-6, 'olive.n.04'),
 (-5, 'camera.n.01'),
 (-5, 'grass.n.01'),
 (-5, 'saltshaker.n.01'),
 (-5, 'soap.n.01'),
 (-4, 'blender.n.01'),
 (-4, 'bucket.n.01'),
 (-3, 'atomizer.n.01'),
 (-3, 'date.n.08'),
 (-3, 'kettle.n.01'),
 (-3, 'lawn.n

In [20]:
# print("\n".join(','.join(synset_requiring_tasks[s]) for s in sorted(legit_task_required_synsets)))
# print("\n".join(','.join(found_task_required_objs_by_synset[s]) for s in sorted(legit_task_required_synsets)))
print("\n".join(','.join(providers[x].split("/")[-1] for x in found_task_required_objs_by_synset[s]) for s in sorted(legit_task_required_synsets)))

legacy_atomizer-tehmzy
legacy_basket-clziqw,legacy_basket-xjftlf,legacy_basket-pqzyxx,legacy_basket-wdsnjy,legacy_basket-wqxrbf,legacy_basket-pcyjbu,legacy_basket-xhdtjn,legacy_basket-pgwdlg,legacy_basket-haewxp
legacy_bathtub-azdgtl,legacy_bathtub-wxrafq,legacy_bathtub-jfhrwj,legacy_bathtub-fdjykf,legacy_bathtub-gulceo,legacy_bathtub-enpxbn,legacy_bathtub-oobbwn

blender-temp,legacy_blender-eyedvd,legacy_blender-cwkvib
coke_bottle-al,legacy_wine_bottle-fzuiwy,flask-fu,sparkling_water-ry,legacy_wine_bottle-vecadn,legacy_water-emquat,coke_bottle-al,decanter-ij,legacy_wine_bottle-hnkiog,legacy_carafe-ocjcgp,baby_bottle-hv,legacy_cruet-njqmqv,lemon_juice_bottle-fa,bottle-wu
legacy_bowl-kthvrl,legacy_bowl-fiarri,trail_mix-hr,shortening-vz,mac_and_cheese-ql,legacy_bowl-zwhewa,legacy_bowl-rlwpcd,legacy_bowl-jfvjep,dog_food-df,legacy_bowl-xpnlup,mashed_potatoes-zw,salad-ur,legacy_bowl-oyidja,cranberries-ll,legacy_bowl-mspdar,legacy_bowl-eipwho,legacy_bowl-fwdfeg,legacy_bowl-adciys,legacy_bowl

In [29]:
# How many objects are task-relevant?
print(f"{len(found_task_required_objects)} relevant objects")
print(found_task_required_objects)

1442 relevant objects
{'bottom_cabinet-immwzb', 'glass-slscza', 'firewood-fyasfx', 'bowl-fwdfeg', 'cup-cpozxi', 'wine_bottle-ivuvbv', 'pedestal_table-ktevfe', 'roasting_pan-nrecsa', 'conference_table-hdomxc', 'desk-juzwaf', 'bowl-npuuir', 'stand-ljtrka', 'stove-yhjzwg', 'teddy_bear-hqlobx', 'lawn-wwoqjw', 'steak-akoxpw', 'wineglass-euzudc', 'saucepot-pkfydm', 'nightstand-uzxrlg', 'dryer-fhtvuq', 'tree-abtcjq', 'coffee_table-tzmkgq', 'wineglass-aakcyj', 'floors-qsesig', 'sofa-hiphpn', 'basket-ceftqq', 'teacup-zdvgol', 'breakfast_table-wwsikw', 'wine_bottle-ynfvhg', 'toy-issvzv', 'erlenmeyer_flask-bpwjxr', 'chopping_board-drjnag', 'carving_knife-oxvnsa', 'display_fridge-wlxyjn', 'floors-mcivwr', 'bowl-jfvjep', 'floors-ljbnpd', 'floors-pmlwwl', 'desk-bvhpnd', 'table_knife-jxdfyy', 'camera-tymxet', 'floors-fzriob', 'tree-olbeqh', 'dreidel-vsmlcg', 'firewood-njlexd', 'table_knife-nvwmkx', 'cup-wfavfd', 'steak-qrkwtt', 'bowl-bnobdx', 'floors-wgqunv', 'grass_patch-kqhokv', 'water_glass-kuiiai

In [36]:
# Get a subsampled version of this such that there is at least 1 thing matching every synset
G2 = nx.Graph()
for s, objs in found_task_required_objs_by_synset.items():
    for obj in objs:
        G2.add_edge(s, obj)
assert nx.bipartite.is_bipartite(G2)
top_set = set(found_task_required_objs_by_synset.keys())
matching = nx.bipartite.maximum_matching(G2, top_set)
cover = nx.bipartite.to_vertex_cover(G2, matching, top_set) & top_set   # TODO: THIS IS INCORRECT!!!
# assert not cover - top_set, str(cover - top_set)
subsampled_found_task_required_objects = set(matching.values()) - top_set

## Load list of objects required by our scenes

In [37]:
# Required by scene
import os
objects_needed_by_scene = {}
for scene, scene_json in zip(available_scenes, scene_object_lists):
    with open(scene_json, "r") as f:
        scene_ol = json.load(f)
    objects_needed_by_scene[scene] = scene_ol["needed_objects"]
all_scene_required_objects = {obj for objs in objects_needed_by_scene.values() for obj in objs}

## Get the combined scene / task object list

In [38]:
b66_required_objects = {
"alarm-dkwmmf",
"apple-qusmpx",
"backpack-gvbiwl",
"bagel-zlxkry",
"ball-nofzds",
"basket-clziqw",
"basket-pgwdlg",
"basket-pqzyxx",
"basket-xjftlf",
"bath_towel-lyqgnf",
"beer-phdimo",
"bow-fhchql",
"bowl-ajzltc",
"bowl-dalyim",
"bowl-eipwho",
"bowl-fedafr",
"bowl-fiarri",
"bowl-fwdfeg",
"bowl-jfvjep",
"bracelet-thqqmo",
"broom-tpyvbt",
"bucket-bdhvnt",
"bucket-lsmlzi",
"bucket-wlilma",
"calculator-kwmmty",
"candle-bhaqam",
"carafe-ocjcgp",
"carton-causya",
"catsup-hvxkso",
"cauldron-zndohl",
"chaise_longue-ajjxcd",
"chaise_longue-xmjcle",
"cheese-hwxeto",
"chest-gbdzls",
"chip-qstxpj",
"christmas_tree_decorated-wakmhn",
"cologne-lyipur",
"container_date-qqddba",
"cookie-dajnau",
"cookie-ggpang",
"cookie-oyhoth",
"cup-ckkwmj",
"cup-nzdsmu",
"cup-rypdvd",
"cup-ykuftq",
"detergent-gkpmii",
"detergent-qjkmhq",
"dishtowel-dtfspn",
"dishtowel-ltydgg",
"dishtowel-qvzbjx",
"document-qkrlsi",
"dustpan-espzwv",
"envelope-urcigc",
"facsimile-mcqqhy",
"fish-qofuoa",
"floor_lamp-jqsuky",
"floor_lamp-lqdmym",
"folder-piezyg",
"folder-poqpux",
"frying_pan-sfbdjn",
"grill-ddlqye",
"gym_shoe-kmcbym",
"gym_shoe-wfmrpw",
"hairbrush-bdylnb",
"hamburger-fyswch",
"hammer-bbzlel",
"hand_towel-jaubic",
"hardback-nubbfj",
"highchair-pahqnp",
"highlighter-tcsmqv",
"jeans-nmvvil",
"kettle-bzisss",
"kiwi-khupqq",
"milk-xbvdpc",
"modem-axqxsv",
"mousetrap-mwfwsv",
"mug-ppzttc",
"newspaper-ukamcl",
"notebook-aanuhi",
"notebook-svisnb",
"oatmeal-zkggxm",
"olive-mhnfdr",
"olive_oil-cqycjk",
"olive_oil-luikop",
"painting-ggloob",
"paper_bag-bzsxgw",
"paper_towel-aclaus",
"parsley-kkgkkd",
"pasta-gwqhko",
"pen-zmimhb",
"pencil-xuanpj",
"plate-iawoof",
"plate-ivbrtz",
"plate-nbuspz",
"plate-qbxfmv",
"plywood-fkmkqa",
"pool-xixlzr",
"pop-bqretx",
"pop-drqbiy",
"pop-ghxeqz",
"pop-mrrozu",
"pop-ubazru",
"pop-vszbvb",
"pop-xlyult",
"pop-xoldze",
"potato-bpwohr",
"pretzel-rmutwn",
"printer-oyqbtq",
"pumpkin-apitmi",
"rag-dgzewy",
"rag-zmmbpz",
"rib-nzbnqp",
"ribbon-puwdwq",
"salad-dhdhul",
"sandal-chgcrm",
"sandal-uhzfis",
"saucepan-fsinsu",
"saw-laysom",
"scanner-juzkjp",
"scraper-icqmje",
"scrub_brush-hsejyi",
"shampoo-dvrzmy",
"shampoo-jccyom",
"shampoo-zoatjo",
"sheet-czdhwu",
"soap-ozifwa",
"sock-rhgweu",
"spaghetti_sauce-zahyds",
"spoon-gpiftn",
"spoon-prbhag",
"sticky_note-tghqep",
"sugar_jar-rfzqei",
"sushi-akpnxn",
"sweater-ekcjci",
"sweater-eknmzl",
"teapot-foaehs",
"tile-kuombe",
"tile-vowkci",
"toothbrush-qbkrzg",
"toothbrush-vkrjps",
"toothpaste-vsclsj",
"toy-aihmqy",
"toy-cobfpg",
"toy-fnueyn",
"toy-tjchdg",
"toy-vcmcoi",
"toy-zuntus",
"trash_can-aefcem",
"trash_can-pdmzhv",
"trash_can-zotrbg",
"tray-spopfj",
"tray-xzcnjq",
"underwear-uwzqxm",
"vacuum-bdmsbr",
"vacuum-wikhik",
"walker-txpihh",
"walker-ykjauh",
"wine_bottle-fzuiwy",
"wrapped_gift-wpmksn",
"wreath-rkrxno",
"armchair-bslhmj",
"armchair-qplklw",
"armchair-tcxiue",
"armchair-vzhxuf",
"armchair-xpcheg",
"basket-clziqw",
"basket-wdsnjy",
"bathtub-fdjykf",
"bed-smmmaf",
"bed-zrumze",
"bottom_cabinet-bamfsz",
"bottom_cabinet-dajebq",
"bottom_cabinet-immwzb",
"bottom_cabinet-jhymlr",
"bottom_cabinet-jrhgeu",
"bottom_cabinet-lwjdmj",
"bottom_cabinet-nddvba",
"bottom_cabinet-qacthv",
"bottom_cabinet-rvpunw",
"bottom_cabinet-slgzfc",
"bottom_cabinet_no_top-bmsclc",
"bottom_cabinet_no_top-pluwfl",
"bottom_cabinet_no_top-qohxjq",
"bottom_cabinet_no_top-qudfwe",
"bottom_cabinet_no_top-spojpj",
"bottom_cabinet_no_top-ufhpbn",
"breakfast_table-dnsjnv",
"breakfast_table-skczfi",
"breakfast_table-uhrsex",
"breakfast_table-zypvuv",
"burner-pmntxh",
"carpet-ctclvd",
"ceilings-ayfjzo",
"ceilings-cbjzgq",
"ceilings-ckxrok",
"ceilings-dashlx",
"ceilings-dslvvl",
"ceilings-dulwkz",
"ceilings-eanlkl",
"ceilings-edrgnz",
"ceilings-ekhjwh",
"ceilings-gbfubh",
"ceilings-gdodun",
"ceilings-gjwnre",
"ceilings-gvaqgm",
"ceilings-gztsev",
"ceilings-hzdavr",
"ceilings-ibqleu",
"ceilings-jfsbfp",
"ceilings-kjpzku",
"ceilings-lfwahr",
"ceilings-lmlzab",
"ceilings-ndlpeo",
"ceilings-odgyya",
"ceilings-oetibq",
"ceilings-ofefox",
"ceilings-pcatii",
"ceilings-phrrml",
"ceilings-qquywv",
"ceilings-rhfker",
"ceilings-rlsnwq",
"ceilings-rnaypg",
"ceilings-rxnnfd",
"ceilings-taubkv",
"ceilings-trhqld",
"ceilings-uahaxk",
"ceilings-vjiuwu",
"ceilings-vnstqh",
"ceilings-vubcbl",
"ceilings-vwwzdu",
"ceilings-vyvqsb",
"ceilings-wvqyit",
"ceilings-wxbmon",
"ceilings-xqauyk",
"ceilings-xusryb",
"ceilings-xzgtpt",
"ceilings-yimfml",
"ceilings-zfnsml",
"ceilings-zlhppj",
"chest-fwstpx",
"coffee_maker-pyttso",
"coffee_table-fqluyq",
"coffee_table-gpkbiw",
"coffee_table-qlmqyy",
"console_table-emeeke",
"countertop-tpuwys",
"dishwasher-dngvvi",
"dishwasher-xlmier",
"dishwasher-znfvmj",
"door-ktydvs",
"door-lvgliq",
"door-ohagsq",
"dryer-xsuyua",
"dryer-zlmnfg",
"floor_lamp-jcbvfi",
"floor_lamp-jqsuky",
"floor_lamp-vdxlda",
"floors-aafddg",
"floors-avvnuv",
"floors-bryjan",
"floors-cborev",
"floors-dclqap",
"floors-dggxhh",
"floors-ekqfhr",
"floors-gjemfr",
"floors-gjpvfd",
"floors-gupykd",
"floors-gwawvk",
"floors-hcqtge",
"floors-ibjsgt",
"floors-ideoae",
"floors-ifmioj",
"floors-jbiwrx",
"floors-jniagb",
"floors-kqgjaj",
"floors-kwnype",
"floors-kxqtap",
"floors-ljbnpd",
"floors-lwmene",
"floors-meryyw",
"floors-mknpoc",
"floors-mowtdm",
"floors-mvnkva",
"floors-nuacoz",
"floors-obpiof",
"floors-pmlwwl",
"floors-ptwlei",
"floors-qhfsxw",
"floors-qrzmkt",
"floors-qsesig",
"floors-qtdpcm",
"floors-rmwqgp",
"floors-rqcmwl",
"floors-sktjer",
"floors-sqtkxe",
"floors-ssjixw",
"floors-suqrjf",
"floors-tcsswg",
"floors-ucjdgj",
"floors-umeasc",
"floors-xzlkei",
"floors-zijaqk",
"floors-znzabl",
"floors-zpfzjw",
"floors-zvyodb",
"fridge-dszchb",
"fridge-xyejdx",
"grandfather_clock-veyqcp",
"laptop-nvulcs",
"loudspeaker-bmpdyv",
"microwave-abzvij",
"microwave-bfbeeb",
"mirror-pevdqu",
"oven-fexqbj",
"oven-wuinhm",
"piano-bnxcvw",
"picture-cltbty",
"picture-etixod",
"picture-fanvpf",
"picture-fhkzlm",
"picture-gwricv",
"picture-hhxttu",
"picture-jpfyrq",
"picture-lucjyq",
"picture-qavxsz",
"picture-qjkajj",
"picture-qtvjzk",
"picture-rciuob",
"picture-szpupz",
"picture-tlwjpv",
"picture-weqggl",
"picture-wfdvzv",
"picture-yrgaal",
"picture-ytxhyl",
"picture-zsirgc",
"pillow-pjqqmb",
"pot_plant-jatssq",
"pot_plant-kxmvco",
"pot_plant-udqjui",
"rail_fence-qmsnld",
"range_hood-iqbpie",
"shelf-njwsoa",
"shelf-owvfik",
"shelf-vgzdul",
"shower-invgma",
"sink-czyfhq",
"sink-ksecxq",
"sink-xiybkb",
"sink-zexzrc",
"sofa-mnfbbh",
"sofa-qnnwfx",
"sofa-uixwiv",
"sofa-xhxdqf",
"speaker_system-snbvop",
"standing_tv-udotid",
"stool-miftfy",
"stool-xcmniq",
"stool-ycfbsd",
"stove-igwqpj",
"stove-rgpphy",
"straight_chair-amgwaw",
"straight_chair-dmcixv",
"straight_chair-eospnr",
"straight_chair-psoizi",
"straight_chair-vkgbbl",
"swivel_chair-qtqitn",
"table_lamp-xbfgjc",
"toilet-kfmkbm",
"toilet-vtqdev",
"toilet-wusctd",
"top_cabinet-dmwxyl",
"top_cabinet-eobsmt",
"top_cabinet-fqhdne",
"top_cabinet-jvdbxh",
"top_cabinet-lsyzkh",
"towel_rack-kqrmrh",
"trash_can-zotrbg",
"wall_mounted_tv-ylvjhb",
"walls-adqmpz",
"walls-afdfgb",
"walls-aigrtq",
"walls-aquzfl",
"walls-axhynp",
"walls-ayjzsb",
"walls-bbqdud",
"walls-bcjmdz",
"walls-bdxwvm",
"walls-bntgiu",
"walls-bqdrep",
"walls-btxdrd",
"walls-bvcaju",
"walls-bvtxbv",
"walls-cesukw",
"walls-cusskx",
"walls-cyflyo",
"walls-dbcxft",
"walls-dekben",
"walls-dfnqnk",
"walls-dondsu",
"walls-dthtvd",
"walls-dzlwbx",
"walls-ejzliv",
"walls-enroos",
"walls-exiopd",
"walls-eyalcx",
"walls-fsaxgs",
"walls-fxibmi",
"walls-gfxhgi",
"walls-gmdnub",
"walls-gtaxam",
"walls-gxvfxw",
"walls-gxynpx",
"walls-hfwelb",
"walls-hmmmqw",
"walls-hnmcoo",
"walls-hnqegn",
"walls-hosciu",
"walls-hqpeoy",
"walls-hsudbq",
"walls-hxakof",
"walls-hyizxy",
"walls-icgqdt",
"walls-icgyhn",
"walls-ifhuaw",
"walls-insggh",
"walls-ioxzum",
"walls-ipzkkp",
"walls-iqbzlu",
"walls-iwgkkg",
"walls-iwipoh",
"walls-jukzkm",
"walls-jvltxt",
"walls-kabquv",
"walls-kvmewy",
"walls-ljngkt",
"walls-llgahv",
"walls-lmdouk",
"walls-mqmdyl",
"walls-ngfxil",
"walls-nmzbqa",
"walls-nthwqa",
"walls-nvtvxi",
"walls-nxxtrq",
"walls-nyvztg",
"walls-odbsjx",
"walls-pefrto",
"walls-pivmuk",
"walls-pmfadr",
"walls-pmqiht",
"walls-prkntu",
"walls-pziwqm",
"walls-qannys",
"walls-qbpprw",
"walls-qcqpyi",
"walls-qisxmx",
"walls-qyejch",
"walls-radpsr",
"walls-rkueec",
"walls-rvvbvi",
"walls-sdxbli",
"walls-snokhn",
"walls-szxayl",
"walls-tjfjwe",
"walls-tmpzcd",
"walls-tsyvvt",
"walls-twxspa",
"walls-ucsylc",
"walls-ugemkf",
"walls-uishdq",
"walls-umjlad",
"walls-uwsqqu",
"walls-vdzmpj",
"walls-vrkkum",
"walls-wcekyw",
"walls-wiihpm",
"walls-wxyhis",
"walls-wycsdd",
"walls-xamxbe",
"walls-xkdbsb",
"walls-xlgydu",
"walls-xrmvpg",
"walls-xwrjsn",
"walls-yeodqe",
"walls-yjbruo",
"walls-yrcjvs",
"walls-ysxqfc",
"walls-zbggqd",
"walls-zlszjq",
"walls-zoonsb",
"walls-zqimeh",
"walls-zssquq",
"washer-omeuop",
"window-ithrgo",
"window-ulnafj",
    }

In [39]:
# Get the right set of files
sr_sources = sorted({providers[x] for x in all_scene_required_objects if x in providers})
print("\n".join(['"' + x.replace('objects/', '') + '",' for x in sr_sources]))

"legacy_armchair-bslhmj",
"legacy_armchair-qplklw",
"legacy_armchair-tcxiue",
"legacy_armchair-vzhxuf",
"legacy_armchair-xpcheg",
"legacy_armchair-xqovyv",
"legacy_basket-clziqw",
"legacy_basket-wdsnjy",
"legacy_bathtub-fdjykf",
"legacy_bed-fnzxxr",
"legacy_bed-mpgavt",
"legacy_bed-smmmaf",
"legacy_bed-zrumze",
"legacy_bottom_cabinet-jrhgeu",
"legacy_bottom_cabinet-lwjdmj",
"legacy_bottom_cabinet-nddvba",
"legacy_bottom_cabinet-nnvyol",
"legacy_bottom_cabinet-nyfebf",
"legacy_bottom_cabinet-qacthv",
"legacy_bottom_cabinet-rntwkg",
"legacy_bottom_cabinet-rvpunw",
"legacy_bottom_cabinet_no_top-bmsclc",
"legacy_bottom_cabinet_no_top-pluwfl",
"legacy_bottom_cabinet_no_top-qohxjq",
"legacy_bottom_cabinet_no_top-qudfwe",
"legacy_bottom_cabinet_no_top-spojpj",
"legacy_bottom_cabinet_no_top-ufhpbn",
"legacy_bottom_cabinet_no_top-vdedzt",
"legacy_bottom_cabinet_no_top-vzzafs",
"legacy_breakfast_table-bmnubh",
"legacy_breakfast_table-dnsjnv",
"legacy_breakfast_table-idnnmz",
"legacy_breakfast_ta

In [40]:
print(all_scene_required_objects - b66_required_objects)
print(b66_required_objects - set(providers.keys()))

{'walls-wzauoy', 'floors-owamxh', 'walls-hukjxe', 'walls-fecefe', 'ceilings-plnpyp', 'walls-xwettc', 'walls-lyuheh', 'ceilings-chcxue', 'ceilings-hjvmxm', 'ceilings-bwcjal', 'ceilings-uidjxd', 'walls-exahpb', 'walls-oyfrsd', 'floors-fzpedl', 'straight_chair-pmpwwi', 'walls-apmsgb', 'floors-mdwoyt', 'floors-bcoszl', 'floors-yoxmsz', 'walls-ijhavy', 'ceilings-hyjmts', 'walls-hdrtcw', 'walls-uvkglh', 'floors-avfwga', 'walls-pcrftt', 'walls-ujdgcj', 'walls-piiojx', 'floors-mcecze', 'ceilings-pueokn', 'ceilings-kdovid', 'walls-beqqbw', 'walls-sfsjho', 'floors-mzywvs', 'floors-wympsq', 'ceilings-ouguhp', 'walls-mlnhxt', 'walls-xumdqs', 'walls-lybykw', 'floors-ozonyy', 'walls-jrmnlu', 'walls-jrrget', 'breakfast_table-bmnubh', 'walls-qvxgfb', 'walls-nfprcv', 'floors-ompijb', 'bottom_cabinet_no_top-vzzafs', 'floors-mcivwr', 'walls-pidgcc', 'floors-fzriob', 'walls-ifeyfb', 'ceilings-gnpckf', 'floors-wgqunv', 'walls-ggligz', 'ceilings-unxwzc', 'floors-txjurc', 'heater-wntwxx', 'walls-ewecsu', 'fl

In [41]:
all_required_objects = b66_required_objects
# all_required_objects = subsampled_found_task_required_objects | all_scene_required_objects | {obj for obj, provider in providers.items() if "legacy" in provider}
print(f"{len(all_scene_required_objects)} objects required by scenes")
print(f"{len(subsampled_found_task_required_objects)} objects required by tasks")
print(f"{len(all_required_objects)} objects required total")

761 objects required by scenes
62 objects required by tasks
491 objects required total


In [42]:
# Get the right set of files
sources = sorted({providers[x] for x in all_required_objects if x in providers})
print("\n".join([f'"{x}",' for x in sources]))

"objects/bratwurst-lt",
"objects/care_products-gt",
"objects/legacy_alarm-dkwmmf",
"objects/legacy_apple-qusmpx",
"objects/legacy_armchair-bslhmj",
"objects/legacy_armchair-qplklw",
"objects/legacy_armchair-tcxiue",
"objects/legacy_armchair-vzhxuf",
"objects/legacy_armchair-xpcheg",
"objects/legacy_backpack-gvbiwl",
"objects/legacy_bagel-zlxkry",
"objects/legacy_ball-nofzds",
"objects/legacy_basket-clziqw",
"objects/legacy_basket-pgwdlg",
"objects/legacy_basket-pqzyxx",
"objects/legacy_basket-wdsnjy",
"objects/legacy_basket-xjftlf",
"objects/legacy_bath_towel-lyqgnf",
"objects/legacy_bathtub-fdjykf",
"objects/legacy_bed-smmmaf",
"objects/legacy_bed-zrumze",
"objects/legacy_beer-phdimo",
"objects/legacy_bottom_cabinet-jrhgeu",
"objects/legacy_bottom_cabinet-lwjdmj",
"objects/legacy_bottom_cabinet-nddvba",
"objects/legacy_bottom_cabinet-qacthv",
"objects/legacy_bottom_cabinet-rvpunw",
"objects/legacy_bottom_cabinet_no_top-bmsclc",
"objects/legacy_bottom_cabinet_no_top-pluwfl",
"objects/l

In [43]:
print(len(sources))

277


## Check how many of the objects have already been successfully exported

In [44]:
successful = 0
tried = 0
for s in sources:
    ems = os.path.join(r"D:\BEHAVIOR-1K\asset_pipeline\cad", s, r"artifacts\export_meshes.success")
    if os.path.exists(ems):
        successful += 1
        
    emj = os.path.join(r"D:\BEHAVIOR-1K\asset_pipeline\cad", s, r"artifacts\export_meshes.json")
    if os.path.exists(emj):
        tried += 1

print(f"Out of {len(sources)} sources")
print(f"Export meshes has been attempted for {tried}")
print(f"Was successful for {successful}")

Out of 277 sources
Export meshes has been attempted for 267
Was successful for 0


## Synset to object for Yunhao

In [21]:
ego_synsets = {"apple": ["apple.n.01"], "backpack": ["backpack.n.01"], "ball": ["ball.n.01", "ball.n.03"], "balloon": ["balloon.n.01"], "banana": ["banana.n.02"], "bathtub": ["bathtub.n.01"], "bed": ["bed.n.01"], "beer": ["beer.n.01"], "belt": ["belt.n.01"], "bicycle": ["bicycle.n.01"], "blender": ["blender.n.01"], "book": ["book.n.01", "book.n.02"], "bookcase": ["bookcase.n.01"], "boot": ["boot.n.01"], "bottle": ["bottle.n.01"], "bowl": ["bowl.n.01", "bowl.n.03"], "box": ["box.n.01"], "bread": ["bread.n.01"], "briefcase": ["briefcase.n.01"], "cake": ["cake.n.03", "cake.n.01"], "calculator": ["calculator.n.02"], "camera": ["camera.n.01"], "candle": ["candle.n.01"], "candy": ["candy.n.01"], "carrot": ["carrot.n.03"], "chair": ["chair.n.01"], "cheese": ["cheese.n.01"], "clock": ["clock.n.01"], "clothing": ["clothing.n.01"], "coat": ["coat.n.01"], "coffee": ["coffee.n.01"], "coin": ["coin.n.01"], "container": ["container.n.01"], "cookie": ["cookie.n.01"], "countertop": ["countertop.n.01"], "cream": ["cream.n.02", "cream.n.03"], "cucumber": ["cucumber.n.02"], "cupboard": ["cupboard.n.01"], "curtain": ["curtain.n.01"], "desk": ["desk.n.01"], "dishwasher": ["dishwasher.n.01"], "doll": ["doll.n.01"], "door": ["door.n.01"], "dress": ["dress.n.01"], "drum": ["drum.n.01"], "duck": ["duck.n.03"], "envelope": ["envelope.n.01"], "eraser": ["eraser.n.01"], "fireplace": ["fireplace.n.01"], "flag": ["flag.n.01", "flag.n.04"], "flashlight": ["flashlight.n.01"], "flower": ["flower.n.01", "flower.n.02"], "food": ["food.n.01", "food.n.02"], "footwear": ["footwear.n.01", "footwear.n.02"], "fork": ["fork.n.01"], "fruit": ["fruit.n.01"], "glove": ["glove.n.02"], "goggles": ["goggles.n.01"], "guitar": ["guitar.n.01"], "hammer": ["hammer.n.02"], "hat": ["hat.n.01"], "heater": ["heater.n.01"], "helmet": ["helmet.n.01"], "jacket": ["jacket.n.01"], "juice": ["juice.n.01"], "kettle": ["kettle.n.01"], "knife": ["knife.n.01"], "ladle": ["ladle.n.01"], "lamp": ["lamp.n.01", "lamp.n.02"], "lantern": ["lantern.n.01"], "laptop": ["laptop.n.01"], "lemon": ["lemon.n.01"], "lipstick": ["lipstick.n.01"], "milk": ["milk.n.01", "milk.n.04"], "mirror": ["mirror.n.01"], "mixer": ["mixer.n.04"], "motorcycle": ["motorcycle.n.01"], "mouse": ["mouse.n.04"], "muffin": ["muffin.n.01"], "mug": ["mug.n.04", "mug.n.01"], "necklace": ["necklace.n.01"], "orange": ["orange.n.01"], "oven": ["oven.n.01"], "paper": ["paper.n.01"], "pear": ["pear.n.01"], "pen": ["pen.n.01"], "pencil": ["pencil.n.01"], "perfume": ["perfume.n.02"], "piano": ["piano.n.01"], "pillow": ["pillow.n.01"], "pineapple": ["pineapple.n.02", "pineapple.n.01"], "pizza": ["pizza.n.01"], "plate": ["plate.n.02", "plate.n.14", "plate.n.04"], "popcorn": ["popcorn.n.02", "popcorn.n.01"], "poster": ["poster.n.01"], "printer": ["printer.n.02", "printer.n.03"], "refrigerator": ["refrigerator.n.01"], "ring": ["ring.n.08"], "sandal": ["sandal.n.01"], "saucer": ["saucer.n.02"], "scale": ["scale.n.07", "scale.n.10"], "scarf": ["scarf.n.01"], "scissors": ["scissors.n.01"], "screwdriver": ["screwdriver.n.01"], "sculpture": ["sculpture.n.01"], "shelf": ["shelf.n.01"], "shirt": ["shirt.n.01"], "sink": ["sink.n.01"], "skateboard": ["skateboard.n.01"], "sock": ["sock.n.01"], "spatula": ["spatula.n.02", "spatula.n.01"], "spoon": ["spoon.n.01"], "stapler": ["stapler.n.01"], "stool": ["stool.n.01"], "strawberry": ["strawberry.n.01"], "sunglasses": ["sunglasses.n.01"], "table": ["table.n.02", "table.n.01"], "tea": ["tea.n.01", "tea.n.05"], "teapot": ["teapot.n.01"], "telephone": ["telephone.n.01"], "toaster": ["toaster.n.02"], "toilet": ["toilet.n.02"], "tomato": ["tomato.n.01"], "tool": ["tool.n.01"], "toothbrush": ["toothbrush.n.01"], "towel": ["towel.n.01"], "treadmill": ["treadmill.n.01"], "tripod": ["tripod.n.01"], "umbrella": ["umbrella.n.01"], "vase": ["vase.n.01"], "violin": ["violin.n.01"], "wallet": ["wallet.n.01"], "wardrobe": ["wardrobe.n.01"], "watch": ["watch.n.01"], "watermelon": ["watermelon.n.02"], "whisk": ["whisk.n.01"], "window": ["window.n.01"], "wine": ["wine.n.01"], "wrench": ["wrench.n.03"], "blanket": ["blanket.n.01"], "condiment": ["condiment.n.01"], "sofa": ["sofa.n.01"]}

In [23]:
from collections import defaultdict
found_ego4d_by_synset = defaultdict(set)
for ego4d, synset_list in ego_synsets.items():
    for s in synset_list:
        # Get the tree of the synset
        child_synsets = [s] + list(nx.descendants(G, s))
        for cs in child_synsets:
            if cs in found_synsets:
                found_task_required_synsets.add(s)  # Only add the sought-after synset, not children

                s_cats = synset_to_cat[cs]
                s_objs = {obj for cat in s_cats for obj in objects_by_category[cat]}
                found_ego4d_by_synset[ego4d].update(s_objs)

In [25]:
dict(found_ego4d_by_synset)

{'apple': {'apple-agveuv', 'apple-omzprq', 'apple-qusmpx', 'apple-sguztn'},
 'backpack': {'backpack-gvbiwl'},
 'ball': {'ball-nofzds',
  'baseball-zanmar',
  'bowling_ball-qzzett',
  'moth_ball-nogvls',
  'moth_ball-nyoucp',
  'punching_bag-svkdji',
  'soccer_ball-ikagkc',
  'softball-erhbze',
  'tennis_ball-rgekxe'},
 'balloon': {'balloon-ixhbmk'},
 'banana': {'banana-vvyyyv'},
 'bathtub': {'bathtub-azdgtl',
  'bathtub-enpxbn',
  'bathtub-fdjykf',
  'bathtub-gulceo',
  'bathtub-jfhrwj',
  'bathtub-oobbwn',
  'bathtub-wxrafq'},
 'bed': {'bed-fnzxxr',
  'bed-gcgodj',
  'bed-guguqb',
  'bed-hkmwyc',
  'bed-hojitx',
  'bed-lslyyi',
  'bed-mpgavt',
  'bed-nwzbyk',
  'bed-pqptex',
  'bed-rhanfx',
  'bed-smmmaf',
  'bed-sunfhf',
  'bed-sztpse',
  'bed-wbjidf',
  'bed-wfxgbb',
  'bed-wyxtqf',
  'bed-xhegju',
  'bed-yghbxh',
  'bed-zrumze',
  'hammock-fglfga',
  'hammock-klhkgd',
  'hammock-lqweda',
  'hammock-qewdqa'},
 'beer': {'beer-cowwjv', 'beer-phdimo'},
 'blender': {'blender-cwkvib', 'b

In [29]:
import json
with open(r"C:\Users\Cem\Downloads\egoobjects_map.json", "w") as f:
    json.dump({x: sorted(y) for x, y in found_ego4d_by_synset.items()}, f, indent=4)